In [ ]:
ls

mlp.ipynb              submission.csv     test.csv   url.txt
sample_submission.csv  submission.gsheet  train.csv


In [ ]:
cd drive/MyDrive/dacon/천체 유형 분류 대회

[Errno 2] No such file or directory: 'drive/MyDrive/dacon/천체 유형 분류 대회'
/content/drive/MyDrive/dacon/천체 유형 분류 대회


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

import sklearn
from sklearn.preprocessing import StandardScaler # 스케일링 함수

In [ ]:
train = pd.read_csv('train.csv', index_col='id')

In [ ]:
train.head()

,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
id,,,,,,,,,,,,,,,,,,,,,,
0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,21.387653,21.572827,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,20.937731,21.063646,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,17.095999,17.076449,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,20.073852,19.794505,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,19.861718,18.810375,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655


In [ ]:
unique_labels = train['type'].unique()  # type 종류 파악
label_dict = {val: i for i, val in enumerate(unique_labels)} # 번호를 주고 dict으로
i2lb = {v:k for k, v in label_dict.items()}

In [ ]:
scaler = StandardScaler()
labels = train['type']
train = train.drop(columns=['fiberID', 'type']) # fiberID는 1000개의 categorical feature이며, 이 커널에서는 무시합니다.

In [ ]:
_mat = scaler.fit_transform(train) # 데이터 변환한건가??
train = pd.DataFrame(_mat, columns=train.columns, index=train.index)

scaler는 fit과 transform 메서드를 지니고 있습니다. fit 메서드로 데이터 변환을 학습하고, transform 메서드로 실제 데이터의 스케일을 조정합니다. 

이때, fit 메서드는 학습용 데이터에만 적용해야 합니다. 그 후, transform 메서드를 학습용 데이터와 테스트 데이터에 적용합니다. scaler는 fit_transform()이란 단축 메서드를 제공합니다. 학습용 데이터에는 fit_transform()메서드를 적용하고, 테스트 데이터에는 transform()메서드를 적용합니다.

In [ ]:
train_x = train
train_y = labels.replace(label_dict)

In [ ]:
# 학습률 설정? 0.0003으로 지정
# Adam 알고리즘을 사용
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0003,
)

https://forensics.tistory.com/28

옵티마이저 (Optimizer)는 손실 함수을 통해 얻은 손실값으로부터 모델을 업데이트하는 방식

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Input(len(train_x.columns)),
  tf.keras.layers.Dense(256*4, activation='elu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(256*4, activation='elu'),
  tf.keras.layers.Dense(256*3, activation='elu'),
  tf.keras.layers.Dropout(0.15),
  tf.keras.layers.Dense(256*3, activation='elu'),
  tf.keras.layers.Dense(256*2, activation='elu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(256*2, activation='elu'),
  tf.keras.layers.Dropout(0.15),
  tf.keras.layers.Dense(256*1, activation='elu'),
  tf.keras.layers.Dense(256*1, activation='elu'),
  tf.keras.layers.Dropout(0.15),
  tf.keras.layers.Dense(128, activation='elu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(64, activation='elu'),
  tf.keras.layers.Dense(32, activation='elu'),
  tf.keras.layers.Dense(19, activation='softmax')
])

In [ ]:
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_x,
          train_y,
          batch_size=256*3,
          validation_split=0.1,
          epochs=200)

Epoch 1/200
235/235 [==============================] - 6s 9ms/step - loss: 1.4891 - accuracy: 0.5590 - val_loss: 0.6654 - val_accuracy: 0.7781
Epoch 2/200
235/235 [==============================] - 2s 7ms/step - loss: 0.7009 - accuracy: 0.7659 - val_loss: 0.5915 - val_accuracy: 0.8080
Epoch 3/200
235/235 [==============================] - 2s 7ms/step - loss: 0.6135 - accuracy: 0.7955 - val_loss: 0.5237 - val_accuracy: 0.8248
Epoch 4/200
235/235 [==============================] - 2s 7ms/step - loss: 0.5709 - accuracy: 0.8101 - val_loss: 0.5041 - val_accuracy: 0.8322
Epoch 5/200
235/235 [==============================] - 2s 7ms/step - loss: 0.5565 - accuracy: 0.8142 - val_loss: 0.4869 - val_accuracy: 0.8353
Epoch 6/200
235/235 [==============================] - 2s 7ms/step - loss: 0.5367 - accuracy: 0.8197 - val_loss: 0.4863 - val_accuracy: 0.8333
Epoch 7/200
235/235 [==============================] - 2s 7ms/step - loss: 0.5258 - accuracy: 0.8219 - val_loss: 0.4787 - val_accuracy: 0.8376

In [ ]:
test = pd.read_csv('test.csv').reset_index(drop=True)
test_ids = test['id']
test = test.drop(columns=['id', 'fiberID'])
test = pd.DataFrame(scaler.transform(test), columns=test.columns, index=test.index)

In [ ]:
pred_mat = model.predict(test)

In [ ]:
sample = pd.read_csv('sample_submission.csv')

submission = pd.DataFrame(pred_mat, index=test.index)
submission = submission.rename(columns=i2lb)
submission = pd.concat([test_ids, submission], axis=1)
submission = submission[sample.columns]
submission.to_csv("submission.csv", index=False)